In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
import pandas as pd

csv_path = r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\merged_all_data.csv"   # 경로 확인해서 수정

df = pd.read_csv(csv_path)

print(df.head())
print("데이터 개수:", len(df))
print("컬럼:", df.columns.tolist())

C:\Users\jang8\AppData\Local\Temp\ipykernel_26340\973179262.py:5: DtypeWarning: Columns (2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


                                  colloquial  answer category disease  \
0      HIV 감염을 의심하고 있는데, 어떤 곳에서 검진을 받아야 할까요?     NaN    감염성질환  HIV 감염   
1  성 접촉 이후 HIV 감염으로 인한 증상이 어떻게 나타나는지 알고 싶어요.     NaN    감염성질환  HIV 감염   
2      HIV 감염을 진단하기 위해 사용되는 주요 검사 방법은 무엇인가요?     NaN    감염성질환  HIV 감염   
3       HIV 감염을 의심할 때 어떤 검사를 통해 진단을 받아야 할까요?     NaN    감염성질환  HIV 감염   
4       HIV 감염 검사는 백혈구 수치 감소로 인해 권장되는 검사인가요?     NaN    감염성질환  HIV 감염   

                                              source source_file subject  \
0  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   
1  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   
2  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   
3  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   
4  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   

  sentence normalized_symptom department  
0      NaN                NaN        NaN  
1      NaN        

In [3]:
import torch
import pandas as pd
import numpy as np
import transformers

print("torch cuda:", torch.cuda.is_available())
print("pandas:", pd.__version__)
print("transformers:", transformers.__version__)

torch cuda: True
pandas: 2.3.3
transformers: 4.57.3


In [4]:
# 학습에 필요한 컬럼만 정리
TEXT_COLUMN = "colloquial"
LABEL_COLUMN = "disease"

data = df[[TEXT_COLUMN, LABEL_COLUMN]].dropna().reset_index(drop=True)

In [5]:
# 라벨 인코딩 (disease → 숫자 ID)
unique_labels = sorted(data[LABEL_COLUMN].unique())
label2id = {lbl: i for i, lbl in enumerate(unique_labels)}
id2label = {i: lbl for lbl, i in label2id.items()}
data["label_id"] = data[LABEL_COLUMN].map(label2id)

In [6]:
# train / val 나누기
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    data,
    test_size=0.1,
    random_state=42,
    stratify=data["label_id"]
)

In [7]:
# KM-BERT 불러오기
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "madatnlp/km-bert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id,
).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at madatnlp/km-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Dataset / DataLoader 만들기
from torch.utils.data import Dataset, DataLoader

TEXT_COLUMN = "colloquial"   # 혹시 다르면 여기만 수정
LABEL_ID_COL = "label_id"    # 앞에서 만들어둔 숫자 라벨 컬럼

MAX_LEN = 128
BATCH_SIZE = 16

class SymptomDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row[TEXT_COLUMN]
        label = int(row[LABEL_ID_COL])

        enc = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt",
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }

train_dataset = SymptomDataset(train_df)
val_dataset   = SymptomDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE)

len(train_dataset), len(val_dataset)

(2311961, 256885)

In [9]:
# 한 배치만 뽑아서 GPU에서 잘 도는지 확인
batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_gpu = {k: v.to(device) for k, v in batch.items()}

model.to(device)
model.eval()
with torch.no_grad():
    out = model(
        input_ids=batch_gpu["input_ids"],
        attention_mask=batch_gpu["attention_mask"],
        labels=batch_gpu["labels"],
    )
print("forward pass OK, loss:", out.loss.item())

input_ids torch.Size([16, 128])
attention_mask torch.Size([16, 128])
labels torch.Size([16])
forward pass OK, loss: 6.3896050453186035


In [10]:
# 학습 루프 세팅 (optimizer / scheduler)
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

EPOCHS = 3
LR = 2e-5

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

num_training_steps = EPOCHS * len(train_loader)
num_warmup_steps = int(0.1 * num_training_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)

print("training steps:", num_training_steps, "warmup:", num_warmup_steps)

training steps: 433494 warmup: 43349


In [11]:
import time, itertools
import numpy as np

# 혹시 장치 변수 없으면 다시 지정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 1) 데이터/배치 크기 확인
print("train_dataset 길이 :", len(train_dataset))
print("val_dataset   길이 :", len(val_dataset))
print("train_loader 배치 수 :", len(train_loader))
print("BATCH_SIZE :", BATCH_SIZE)

# 2) 배치 20개만 돌려서 시간 측정
num_test_steps = 20  # 필요하면 10이나 30으로 바꿔도 됨

start = time.time()

with torch.no_grad():
    for step, batch in enumerate(itertools.islice(train_loader, num_test_steps), start=1):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )
        if step % 5 == 0:
            print(f"{step}/{num_test_steps} step 완료")

end = time.time()
elapsed = end - start

print("\n배치", num_test_steps, "개 도는 데 걸린 시간:", f"{elapsed:.2f}", "초")
print("배치 1개당 평균 시간:", f"{elapsed/num_test_steps:.3f}", "초")

# 3) 이 속도 기준으로 1 epoch 예상 시간 출력
steps_per_epoch = len(train_loader)
estimated_epoch_time = (elapsed / num_test_steps) * steps_per_epoch
print("\n[추정] 1 epoch 예상 시간:", f"{estimated_epoch_time/60:.2f}", "분")

train_dataset 길이 : 2311961
val_dataset   길이 : 256885
train_loader 배치 수 : 144498
BATCH_SIZE : 16
5/20 step 완료
10/20 step 완료
15/20 step 완료
20/20 step 완료

배치 20 개 도는 데 걸린 시간: 1.09 초
배치 1개당 평균 시간: 0.054 초

[추정] 1 epoch 예상 시간: 130.86 분


In [12]:
save_dir = "./kmbert_checkpoint_longrun"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("저장 완료:", save_dir)

저장 완료: ./kmbert_checkpoint_longrun


In [23]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from tqdm.auto import tqdm

# ------------------------------
# 평가 함수
# ------------------------------
def evaluate(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            all_labels.extend(batch["labels"].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    acc = accuracy_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds, average="macro")
    return acc, f1


# ------------------------------
# 학습 시작
# ------------------------------
EPOCHS = 1       # 🔥 핵심: 1 epoch만!

model.to(device)
LOG_EVERY = 1000 # 로그 출력 간격

print("🔥 KM-BERT 1 Epoch 학습 시작!")

for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_loss = 0.0

    print(f"\n===== EPOCH {epoch}/{EPOCHS} 시작 =====")

    pbar = tqdm(enumerate(train_loader, start=1),
                total=len(train_loader),
                desc=f"Epoch {epoch}")

    for step, batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
        )

        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()

        # tqdm에 실시간 loss 표시
        pbar.set_postfix({"loss": loss.item()})

        # 콘솔 출력 로그
        if step % LOG_EVERY == 0:
            print(f"[epoch {epoch}] step {step}/{len(train_loader)}  loss={loss.item():.4f}")

    # epoch 끝나고 평가
    avg_train_loss = epoch_loss / len(train_loader)
    val_acc, val_f1 = evaluate(model, val_loader)

    print(f"\n[EPOCH {epoch}] "
          f"train_loss={avg_train_loss:.4f} | "
          f"val_acc={val_acc:.4f} | "
          f"val_macro_f1={val_f1:.4f}\n")

print("🎉 1 epoch 학습 완료!")

🔥 KM-BERT 1 Epoch 학습 시작!

===== EPOCH 1/1 시작 =====


Epoch 1:   0%|          | 0/144498 [00:00<?, ?it/s]

[epoch 1] step 1000/144498  loss=0.0092
[epoch 1] step 2000/144498  loss=0.0384
[epoch 1] step 3000/144498  loss=0.1773
[epoch 1] step 4000/144498  loss=0.0035
[epoch 1] step 5000/144498  loss=0.0187
[epoch 1] step 6000/144498  loss=0.0062
[epoch 1] step 7000/144498  loss=0.0679
[epoch 1] step 8000/144498  loss=0.0388
[epoch 1] step 9000/144498  loss=0.0197
[epoch 1] step 10000/144498  loss=0.0222
[epoch 1] step 11000/144498  loss=0.0757
[epoch 1] step 12000/144498  loss=0.0023
[epoch 1] step 13000/144498  loss=0.0029
[epoch 1] step 14000/144498  loss=0.0559
[epoch 1] step 15000/144498  loss=0.1410
[epoch 1] step 16000/144498  loss=0.0164
[epoch 1] step 17000/144498  loss=0.1808
[epoch 1] step 18000/144498  loss=0.1062
[epoch 1] step 19000/144498  loss=0.0104
[epoch 1] step 20000/144498  loss=0.3693
[epoch 1] step 21000/144498  loss=0.0015
[epoch 1] step 22000/144498  loss=0.0886
[epoch 1] step 23000/144498  loss=0.1592
[epoch 1] step 24000/144498  loss=0.0017
[epoch 1] step 25000/1444

In [25]:
save_dir = "./kmbert_epoch1"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("저장 완료:", save_dir)

저장 완료: ./kmbert_epoch1


In [26]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    pred_id = torch.argmax(outputs.logits, dim=-1).item()
    return id2label[pred_id]

print(predict("배가 너무 아프고 토할 것 같아요"))

복수


In [29]:
print(predict("머리가 깨질 것처럼 아파요"))
print(predict("목이 너무 아파요"))
print(predict("설사가 계속 나요"))
print(predict("메스꺼워서 토할 것 같아요"))

두통
상기도 감염
바이러스성 장염
양수색전증


In [30]:
inputs = tokenizer("배가 너무 아프고 토할 것 같아요", return_tensors="pt").to("cuda")
outputs = model(**inputs)
print(torch.argmax(outputs.logits, dim=-1).item())

241


In [31]:
print(label2id["복수"])

241


In [34]:
import torch
import torch.nn.functional as F

def predict_topk(text, k=5):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)[0]
    topk = torch.topk(probs, k)
    
    for score, idx in zip(topk.values.tolist(), topk.indices.tolist()):
        print(f"{id2label[idx]}  ({score:.4f})")

predict_topk("배가 너무 아프고 토할 것 같아요", k=5)

복수  (0.2143)
폐결핵  (0.1982)
덤핑 증후군  (0.1476)
복막염  (0.0930)
신장 결석  (0.0420)


In [35]:
def predict_with_conf(text, k=5, min_conf=0.5, margin=0.1):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)[0]
    topk = torch.topk(probs, k)
    
    top_probs = topk.values.tolist()
    top_ids   = topk.indices.tolist()
    top_labels = [id2label[i] for i in top_ids]

    # 1) 전체적으로 확신이 낮으면 → "불확실"
    if top_probs[0] < min_conf or (top_probs[0] - top_probs[1]) < margin:
        return {
            "label": None,          # 확정 X
            "reason": "low_conf",
            "candidates": list(zip(top_labels, top_probs)),
        }
    else:
        return {
            "label": top_labels[0], # 확정
            "prob": top_probs[0]
        }

result = predict_with_conf("배가 너무 아프고 토할 것 같아요")
print(result)

{'label': None, 'reason': 'low_conf', 'candidates': [('복수', 0.21432791650295258), ('폐결핵', 0.19823548197746277), ('덤핑 증후군', 0.14763283729553223), ('복막염', 0.09296512603759766), ('신장 결석', 0.042047109454870224)]}


In [36]:
test_sentences = [
    "머리가 깨질 것처럼 아파요",          # 두통
    "어지럽고 핑 도는 느낌이에요",        # 어지럼
    "목이 너무 따갑고 아파요",            # 인후통/상기도
    "기침이 계속 나고 가래도 있어요",      # 기침/호흡기
    "코가 막히고 콧물이 계속 나요",       # 비염/감기
    "속이 메스껍고 토할 것 같아요",        # 메스꺼움/구토
    "배가 콕콕 찌르듯이 아파요",          # 복통
    "설사를 하루에 여러 번 하고 있어요",   # 설사
    "변을 못 보고 배가 더부룩해요",        # 변비
    "온몸이 쑤시고 근육통이 심해요",        # 근육통
]

def predict_top1(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)[0]
    pred_id = torch.argmax(probs).item()
    return id2label[pred_id], probs[pred_id].item()

for s in test_sentences:
    label, p = predict_top1(s)
    print(f"[입력] {s}")
    print(f" -> 예측: {label}  (conf={p:.3f})\n")

[입력] 머리가 깨질 것처럼 아파요
 -> 예측: 두통  (conf=0.685)

[입력] 어지럽고 핑 도는 느낌이에요
 -> 예측: 저혈압  (conf=0.990)

[입력] 목이 너무 따갑고 아파요
 -> 예측: 인후두염  (conf=0.751)

[입력] 기침이 계속 나고 가래도 있어요
 -> 예측: 만성 기침  (conf=0.348)

[입력] 코가 막히고 콧물이 계속 나요
 -> 예측: 만성 비염  (conf=0.778)

[입력] 속이 메스껍고 토할 것 같아요
 -> 예측: 덤핑 증후군  (conf=0.560)

[입력] 배가 콕콕 찌르듯이 아파요
 -> 예측: 복부 통증  (conf=0.772)

[입력] 설사를 하루에 여러 번 하고 있어요
 -> 예측: 바이러스성 장염  (conf=0.958)

[입력] 변을 못 보고 배가 더부룩해요
 -> 예측: 변비  (conf=0.978)

[입력] 온몸이 쑤시고 근육통이 심해요
 -> 예측: 부갑상선기능항진증  (conf=0.226)



In [42]:
import random
import torch
import torch.nn.functional as F

model.eval()
errors = []

# 1) 텍스트가 비어있지 않고, NaN 아닌 행만 필터링
valid_df = df[df["colloquial"].notna()].copy()        # NaN 제거
valid_df = valid_df[valid_df["colloquial"] != ""]     # 빈 문자열 제거

# 2) 그 중에서 100개만 랜덤 샘플
sample_df = valid_df.sample(n=100, random_state=42)

texts = sample_df["colloquial"].astype(str).tolist()   # 전부 str로 변환
gold_labels = sample_df["disease"].tolist()

with torch.no_grad():
    for text, gold_label in zip(texts, gold_labels):
        # 혹시 모를 이상값 한 번 더 방어
        if not isinstance(text, str) or text.strip() == "":
            continue

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
        ).to(device)

        outputs = model(**inputs)
        pred_id = torch.argmax(outputs.logits, dim=-1).item()
        pred_label = id2label[pred_id]

        if pred_label != gold_label:
            errors.append((text, gold_label, pred_label))

print("틀린 샘플 개수:", len(errors))

for sent, gold, pred in random.sample(errors, min(10, len(errors))):
    print("-" * 60)
    print("문장 :", sent)
    print("정답 :", gold)
    print("예측 :", pred)

틀린 샘플 개수: 3
------------------------------------------------------------
문장 : 쿠싱 증후군이 발생하는 이유와 그로 인한 증상에 대해 알려주세요.
정답 : 틱 장애
예측 : 쿠싱 증후군
------------------------------------------------------------
문장 : 부갑상선기능저하증이라는 질병에 대해 자세한 정보를 알려주세요.
정답 : 갑상선기능저하증
예측 : 부갑상선기능저하증
------------------------------------------------------------
문장 : 당뇨병의 핵심적인 특징과 증상에 대해 알고 싶어요.
정답 : 당뇨망막병증
예측 : 당뇨병


In [41]:
from sklearn.metrics import classification_report

all_labels = []
all_preds  = []

model.eval()
with torch.no_grad():
    for batch in val_loader:
        labels = batch["labels"].cpu().numpy()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )
        preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

        all_labels.extend(labels)
        all_preds.extend(preds)

# ✅ 실제로 등장한 라벨만 사용
unique_label_ids = sorted(set(all_labels) | set(all_preds))
target_names = [id2label[i] for i in unique_label_ids]

print("라벨 개수 (val에서 실제 등장한 것만):", len(unique_label_ids))

print(
    classification_report(
        all_labels,
        all_preds,
        labels=unique_label_ids,      # <- 이게 중요
        target_names=target_names,    # <- 여기는 그에 맞춰 이름만
        digits=4
    )
)

라벨 개수 (val에서 실제 등장한 것만): 573
                  precision    recall  f1-score   support

          HIV 감염     0.9873    0.9835    0.9854       789
         VDT 증후군     0.9974    0.9923    0.9948       388
           가와사키병     0.9946    0.9982    0.9964       558
             각막염     0.9949    0.9983    0.9966       582
            간 전이     0.9938    0.9969    0.9953       320
             간경화     0.9957    0.9585    0.9767       482
             간농양     0.9942    1.0000    0.9971       343
           간성 뇌증     0.9692    1.0000    0.9843       440
         간세포성 암종     0.9727    0.9920    0.9822       251
              간염     0.8971    0.7897    0.8400      1921
       간의 양성 신생물     0.9954    0.9954    0.9954       219
         갈락토오스혈증     1.0000    0.9913    0.9957       231
           갈색세포종     0.9554    0.9885    0.9717       347
          갑상선 결절     0.9955    0.9911    0.9933       449
        갑상선기능저하증     0.9581    0.9295    0.9436       837
        갑상선기능항진증     0.9641    0.8447    0

In [60]:
import torch
import torch.nn.functional as F

# -----------------------------------------
# 1) 신뢰도 기준
# -----------------------------------------
LOW_CONF_THRESHOLD = 0.5       # top-1 확률이 이거 이하이면 "신뢰도 낮음"
MARGIN_THRESHOLD   = 0.1       # 1등과 2등 확률 차이가 이거보다 작으면 애매하다고 봄

# -----------------------------------------
# 2) 너무 구체적인 질환 -> 상위 개념으로 매핑
# -----------------------------------------
TOO_SPECIFIC_MAP = {
    "덤핑 증후군": "구토",
    "양수색전증": "구토",
    "복수": "복부 통증",
    "구강섬유종": "근육통",
    "당뇨망막병증": "당뇨병",
}

# -----------------------------------------
# 3) 산과/임신 관련 라벨 후보들
# -----------------------------------------
OB_LABELS = {
    "양수색전증",
    "자간전증",
    "태반 조기 박리",
    "임신성 고혈압",
    "임신성 당뇨병",
}
OB_KEYWORDS = ["임신", "임산부", "출산", "유산", "양수", "태아", "산모"]

# -----------------------------------------
# 4) GI(소화기) 쪽 “안전한” 라벨만 모아서 사용
# -----------------------------------------
GI_SAFE_LABELS = {
    "급성 위장염",
    "기능성 위장 장애",
    "노로바이러스 장염",
    "바이러스성 장염",
    "염증성 장질환",
    "소화불량",
    "소화성궤양",
    "복부 통증",
    "장염",
}
GI_KEYWORDS = ["토", "구토", "메스꺼", "속이", "위", "장", "설사", "배가 아프"]

# -----------------------------------------
# 5) 통증/근육통 관련 라벨 & 키워드
# -----------------------------------------
PAIN_LABELS = {
    "골관절염",
    "통풍성 관절염",
    "요추 추간판 탈출증",
    "요추관 협착증",
    "좌골신경통",
}
PAIN_KEYWORDS = ["쑤시", "근육통", "몸살", "아파요", "통증", "관절", "허리", "무릎", "어깨"]

# -----------------------------------------
# 최종 예측 함수
# -----------------------------------------
def predict_with_rules(text, k=5):
    # top-k logits 가져오기
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(device)

    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)[0]
    topk = torch.topk(probs, k)

    top_indices = topk.indices.tolist()
    top_probs   = topk.values.tolist()
    top_labels  = [id2label[i] for i in top_indices]

    raw_label = top_labels[0]
    raw_prob  = float(top_probs[0])

    final_label = raw_label
    final_prob  = raw_prob
    reasons = []

    # -----------------------------------------
    # (1) 신뢰도 체크
    # -----------------------------------------
    low_conf = False
    if raw_prob < LOW_CONF_THRESHOLD:
        low_conf = True
        reasons.append("low_conf")
    if len(top_probs) > 1 and (top_probs[0] - top_probs[1]) < MARGIN_THRESHOLD:
        low_conf = True
        reasons.append("small_margin")

    # -----------------------------------------
    # (2) 너무 구체적인 질환은 상위 개념으로 매핑
    # -----------------------------------------
    if final_label in TOO_SPECIFIC_MAP:
        old = final_label
        final_label = TOO_SPECIFIC_MAP[final_label]
        reasons.append(f"mapped_from:{old}->{final_label}")

    # -----------------------------------------
    # (3) 산과/임신 관련 규칙
    #     → 문장에 임신/양수 등 키워드가 있고, 현재 라벨이 산과 라벨이 아니며,
    #       top-k 안에 산과 라벨이 있을 때만 교체
    # -----------------------------------------
    if any(kw in text for kw in OB_KEYWORDS):
        if final_label not in OB_LABELS:
            for lbl, prob in zip(top_labels, top_probs):
                if lbl in OB_LABELS:
                    reasons.append(f"OB_rule -> choose:{lbl}")
                    final_label = lbl
                    final_prob  = float(prob)
                    break

    # -----------------------------------------
    # (4) GI(소화기) 규칙
    #   - 문장에 GI 키워드가 있고
    #   - 신뢰도가 낮을 때만(low_conf=True)
    #   - top-k 안에 GI_SAFE_LABELS 중 하나가 있을 때 그걸로 교체
    #   → 이렇게 해야 엉뚱한 문장까지 전부 복부 통증으로 가는 걸 막을 수 있음
    # -----------------------------------------
    if any(kw in text for kw in GI_KEYWORDS):
        gi_candidates = [
            (lbl, prob) for lbl, prob in zip(top_labels, top_probs)
            if lbl in GI_SAFE_LABELS
        ]
        if low_conf and gi_candidates and final_label not in GI_SAFE_LABELS:
            lbl, prob = gi_candidates[0]  # top-k 순서 그대로라 0번째가 제일 확률 높음
            reasons.append(f"GI_rule -> choose:{lbl}")
            final_label = lbl
            final_prob  = float(prob)

    # -----------------------------------------
    # (5) 통증/근육통 관련 규칙
    #   - 문장에 통증 키워드가 있고
    #   - 신뢰도가 낮을 때만, top-k에서 PAIN_LABELS 중 하나를 선택
    # -----------------------------------------
    if any(kw in text for kw in PAIN_KEYWORDS):
        pain_candidates = [
            (lbl, prob) for lbl, prob in zip(top_labels, top_probs)
            if lbl in PAIN_LABELS
        ]
        if low_conf and pain_candidates and final_label not in PAIN_LABELS:
            lbl, prob = pain_candidates[0]
            reasons.append(f"PAIN_rule -> choose:{lbl}")
            final_label = lbl
            final_prob  = float(prob)

    # -----------------------------------------
    # 결과 반환
    # -----------------------------------------
    return {
        "input_text": text,
        "raw": (raw_label, raw_prob),
        "final": (final_label, final_prob),
        "low_conf": low_conf,
        "reasons": reasons,
        "top5": list(zip(top_labels, [float(p) for p in top_probs])),
    }

In [62]:
tests = [
    "배가 너무 아프고 토할 것 같아요",
    "메스꺼워서 토할 것 같아요",
    "온몸이 쑤시고 근육통이 심해요",
]
for t in tests:
    r = predict_with_rules(t, k=20)
    print("입력 :", r["input_text"])
    print(" raw :", r["raw"])
    print("final:", r["final"])
    print("low_conf:", r["low_conf"])
    print("reasons:", r["reasons"])
    print("-"*70)

입력 : 배가 너무 아프고 토할 것 같아요
 raw : ('복수', 0.21432791650295258)
final: ('복부 통증', 0.21432791650295258)
low_conf: True
reasons: ['low_conf', 'small_margin', 'mapped_from:복수->복부 통증']
----------------------------------------------------------------------
입력 : 메스꺼워서 토할 것 같아요
 raw : ('양수색전증', 0.07995087653398514)
final: ('구토', 0.07995087653398514)
low_conf: True
reasons: ['low_conf', 'small_margin', 'mapped_from:양수색전증->구토']
----------------------------------------------------------------------
입력 : 온몸이 쑤시고 근육통이 심해요
 raw : ('부갑상선기능항진증', 0.2259724736213684)
final: ('골관절염', 0.016293829306960106)
low_conf: True
reasons: ['low_conf', 'small_margin', 'PAIN_rule -> choose:골관절염']
----------------------------------------------------------------------


In [47]:
list(id2label.values())

['B형간염',
 'C형간염',
 'HIV 감염',
 'VDT 증후군',
 '가와사키병',
 '각막염',
 '간 전이',
 '간경화',
 '간농양',
 '간성 뇌증',
 '간세포성 암종',
 '간암',
 '간염',
 '간의 양성 신생물',
 '갈락토오스혈증',
 '갈색세포종',
 '갑상선 결절',
 '갑상선기능저하증',
 '갑상선기능항진증',
 '갑상선암',
 '갑상선염',
 '강박 장애',
 '강직성 척추염',
 '거북목 증후군',
 '건강염려증',
 '건선',
 '건초염',
 '결막염',
 '결절종',
 '결핵',
 '경추 추간판 탈출증',
 '고나트륨혈증',
 '고마그네슘혈증',
 '고막염',
 '고산병',
 '고칼륨혈증',
 '고칼슘혈증',
 '고혈압',
 '고환 종양',
 '고환염',
 '골감소증',
 '골관절염',
 '골다공증',
 '골반 염증성 질환',
 '골수염',
 '골수증식종양',
 '골수형성이상증후군',
 '골육종',
 '골형성부전증',
 '곰팡이 감염',
 '공기가슴증(기흉)',
 '공황 장애',
 '과다월경',
 '과민성 방광',
 '과숙아',
 '과호흡 증후군',
 '관절염',
 '광견병',
 '광우병',
 '괴혈병',
 '교모세포종',
 '구강암',
 '구개열',
 '구루병',
 '구순열',
 '구순염',
 '구순포진',
 '구획 증후군',
 '궤양성 대장염',
 '근감소증',
 '근긴장성 이영양증',
 '근시',
 '금단 현상',
 '급성 A형 간염',
 '급성 B형 간염',
 '급성 C형 간염',
 '급성 기관지염',
 '급성 담낭염',
 '급성 류마티스성 열',
 '급성 바이러스성 간염',
 '급성 방광염',
 '급성 신부전',
 '급성 신우신염',
 '급성 심근경색증',
 '급성 위장염',
 '급성 중이염',
 '급성 질염',
 '급성 충수염',
 '급성 췌장염',
 '급성골수성백혈병',
 '급성림프모구백혈병',
 '기관지폐 형성이상',
 '기능성 위장 장애',
 '기도 폐쇄',
 '기면증',
 '기미',
 '기억상실증',
 

In [56]:
# val_df 안에 'colloquial'(구어체), 'disease'(정답 라벨) 컬럼이 있다고 가정
val_texts  = val_df["colloquial"].tolist()
val_labels = [label2id[y] for y in val_df["disease"]]  # 정답을 id로 변환

print("val 개수:", len(val_texts), len(val_labels))

val 개수: 256885 256885


In [57]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm.auto import tqdm
import numpy as np

def eval_plain(model, texts, labels):
    model.eval()
    all_labels = []
    all_preds  = []

    with torch.no_grad():
        for text, gold_id in tqdm(zip(texts, labels), total=len(texts)):
            inputs = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                padding=True
            ).to(device)

            outputs = model(**inputs)
            pred_id = torch.argmax(outputs.logits, dim=-1).item()

            all_labels.append(gold_id)
            all_preds.append(pred_id)

    all_labels = np.array(all_labels)
    all_preds  = np.array(all_preds)

    acc = accuracy_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds, average="macro")

    print(f"[PLAIN] val_acc={acc:.4f} | val_macro_f1={f1:.4f}")
    return all_labels, all_preds

plain_y, plain_pred = eval_plain(model, val_texts, val_labels)

  0%|          | 0/256885 [00:00<?, ?it/s]

[PLAIN] val_acc=0.9785 | val_macro_f1=0.9786


In [59]:
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm
import numpy as np

def eval_with_rules(model, texts, labels, k=20, max_error_show=10):
    model.eval()
    all_labels  = []
    all_preds   = []
    error_cases = []
    skipped     = 0   # fallback도 못 한 케이스 수

    for text, gold_id in tqdm(zip(texts, labels), total=len(texts)):
        r = predict_with_rules(text, k=k)

        # r 구조: 
        #   "raw"   -> (raw_label_str,  raw_prob_float)
        #   "final" -> (final_label_str, final_prob_float)
        final_label, final_prob = r["final"]
        raw_label,   raw_prob   = r["raw"]

        # 1) 우선 final_label 시도
        if final_label in label2id:
            pred_id = label2id[final_label]

        # 2) final_label이 새로운 라벨이면 raw_label로 fallback
        elif raw_label in label2id:
            pred_id = label2id[raw_label]

        # 3) 둘 다 없으면 이 샘플은 평가에서 스킵
        else:
            skipped += 1
            continue

        all_labels.append(gold_id)
        all_preds.append(pred_id)

        # 에러 케이스 수집 (보기용)
        if pred_id != gold_id and len(error_cases) < max_error_show:
            error_cases.append(
                (text, id2label[gold_id], 
                 id2label[pred_id],    # 실제 사용한 라벨 (final or raw)
                 r["raw"][0],          # raw 라벨이 뭐였는지
                 r["final"][0],        # final 라벨이 뭐였는지
                 r["reasons"])
            )

    all_labels = np.array(all_labels)
    all_preds  = np.array(all_preds)

    acc = accuracy_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds, average="macro")

    print(f"[RULE]  val_acc={acc:.4f} | val_macro_f1={f1:.4f}")
    print(f"평가에서 스킵된 샘플 수: {skipped}")
    print(f"에러 샘플 {len(error_cases)}개 (최대 {max_error_show}개만 출력)\n")

    for sent, gold, used_pred, raw_lbl, final_lbl, reasons in error_cases:
        print("-" * 60)
        print("문장:", sent)
        print("정답:", gold)
        print("실제 사용된 예측:", used_pred)
        print("raw_label:", raw_lbl, "| final_label:", final_lbl)
        print("reasons:", reasons)

    return all_labels, all_preds, error_cases

# 다시 실행
rule_y, rule_pred, rule_errors = eval_with_rules(model, val_texts, val_labels, k=20)

  0%|          | 0/256885 [00:00<?, ?it/s]

[RULE]  val_acc=0.8465 | val_macro_f1=0.8800
평가에서 스킵된 샘플 수: 0
에러 샘플 10개 (최대 10개만 출력)

------------------------------------------------------------
문장: 반사회성 인격장애의 정확한 정의를 알고 싶어요.
정답: 반사회성 인격장애
실제 사용된 예측: 급성 위장염
raw_label: 반사회성 인격장애 | final_label: 급성 위장염
reasons: ['GI_rule -> choose:급성 위장염']
------------------------------------------------------------
문장: 망막 박리의 위험성을 줄이기 위해 라섹이나 라식을 선택하는 것이 좋을까요?
정답: 근시
실제 사용된 예측: 기능성 위장 장애
raw_label: 근시 | final_label: 기능성 위장 장애
reasons: ['GI_rule -> choose:기능성 위장 장애']
------------------------------------------------------------
문장: 고혈압을 진단하기 위해 뒷골이 아플 때 어떤 검사를 받아봐야 할까요?
정답: 고혈압
실제 사용된 예측: 복부 통증
raw_label: 고혈압 | final_label: 복부 통증
reasons: ['GI_rule -> choose:복부 통증']
------------------------------------------------------------
문장: 사시 증상을 치료하기 위해 어떤 치료 방법이 효과적인지 알고 싶어요.
정답: 사시
실제 사용된 예측: 기능성 위장 장애
raw_label: 사시 | final_label: 기능성 위장 장애
reasons: ['GI_rule -> choose:기능성 위장 장애']
------------------------------------------------------------
문장: 포경 진단을 받기 위해 방문

In [64]:
# gpt 피셜 이 함수 기준으로 api 만들면 된다네요
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = "./kmbert_epoch1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
model.eval()

def infer_symptom(text: str):
    """
    input: 한국어 구어체 증상 문장(str)
    output: {
        "raw_label": str,
        "raw_prob": float,
        "final_label": str,
        "final_prob": float,
        "low_conf": bool,
        "reasons": list[str]
    }
    """
    r = predict_with_rules(text, k=20)  # 네가 만든 함수 그대로 사용
    return {
        "raw_label":  r["raw"][0],
        "raw_prob":   float(r["raw"][1]),
        "final_label":r["final"][0],
        "final_prob": float(r["final"][1]),
        "low_conf":   bool(r["low_conf"]),
        "reasons":    list(r["reasons"]),
    }

print(infer_symptom("배가 너무 아프고 토할 것 같아요"))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'raw_label': '복수', 'raw_prob': 0.21432791650295258, 'final_label': '복부 통증', 'final_prob': 0.21432791650295258, 'low_conf': True, 'reasons': ['low_conf', 'small_margin', 'mapped_from:복수->복부 통증']}


In [66]:
all_labels = list(id2label.values())
len(all_labels), all_labels

(583,
 ['B형간염',
  'C형간염',
  'HIV 감염',
  'VDT 증후군',
  '가와사키병',
  '각막염',
  '간 전이',
  '간경화',
  '간농양',
  '간성 뇌증',
  '간세포성 암종',
  '간암',
  '간염',
  '간의 양성 신생물',
  '갈락토오스혈증',
  '갈색세포종',
  '갑상선 결절',
  '갑상선기능저하증',
  '갑상선기능항진증',
  '갑상선암',
  '갑상선염',
  '강박 장애',
  '강직성 척추염',
  '거북목 증후군',
  '건강염려증',
  '건선',
  '건초염',
  '결막염',
  '결절종',
  '결핵',
  '경추 추간판 탈출증',
  '고나트륨혈증',
  '고마그네슘혈증',
  '고막염',
  '고산병',
  '고칼륨혈증',
  '고칼슘혈증',
  '고혈압',
  '고환 종양',
  '고환염',
  '골감소증',
  '골관절염',
  '골다공증',
  '골반 염증성 질환',
  '골수염',
  '골수증식종양',
  '골수형성이상증후군',
  '골육종',
  '골형성부전증',
  '곰팡이 감염',
  '공기가슴증(기흉)',
  '공황 장애',
  '과다월경',
  '과민성 방광',
  '과숙아',
  '과호흡 증후군',
  '관절염',
  '광견병',
  '광우병',
  '괴혈병',
  '교모세포종',
  '구강암',
  '구개열',
  '구루병',
  '구순열',
  '구순염',
  '구순포진',
  '구획 증후군',
  '궤양성 대장염',
  '근감소증',
  '근긴장성 이영양증',
  '근시',
  '금단 현상',
  '급성 A형 간염',
  '급성 B형 간염',
  '급성 C형 간염',
  '급성 기관지염',
  '급성 담낭염',
  '급성 류마티스성 열',
  '급성 바이러스성 간염',
  '급성 방광염',
  '급성 신부전',
  '급성 신우신염',
  '급성 심근경색증',
  '급성 위장염',
  '급성 중이염',
  '급성 질염',
  '급성 충수염',
  '급성 췌장

In [67]:
# 0) 전체 라벨 가져오기 (이미 있다면 이 줄은 생략해도 됨)
all_labels = list(id2label.values())

LABEL2DEPT = {}
LABEL2SEVERITY = {}

for lbl in all_labels:
    # 기본값
    dept = "내과"
    severity = "일반"

    # -------------------------
    # 1) 정신건강의학과
    # -------------------------
    if any(kw in lbl for kw in [
        "우울증", "불안", "공황", "조현병", "양극성", "인격장애",
        "주의력 결핍", "주의력결핍", "자폐", "틱 장애", "섭식 장애",
        "행동 및 충동", "망상 장애", "해리 장애", "외상 후 스트레스",
        "알코올 의존성", "인터넷 중독", "성기능장애", "금단 현상"
    ]):
        dept = "정신건강의학과"

    # -------------------------
    # 2) 신경과 / 신경외과
    # -------------------------
    elif any(kw in lbl for kw in [
        "두통", "편두통", "뇌", "경련", "발작",
        "파킨슨병", "알츠하이머", "치매", "실어증",
        "뇌전증", "척수", "근위축", "근무력증", "말초신경병증",
    ]):
        dept = "신경과"

    # -------------------------
    # 3) 심장/순환기내과
    # -------------------------
    elif any(kw in lbl for kw in [
        "심근경색", "협심증", "허혈성 심질환", "부정맥",
        "심방 세동", "심방 조동", "심실 빈맥",
        "승모판", "대동맥", "심낭염", "심근염", "심부전",
        "심계항진", "혈전색전증", "죽상경화증"
    ]):
        dept = "순환기내과"

    # -------------------------
    # 4) 폐/호흡기내과
    # -------------------------
    elif any(kw in lbl for kw in [
        "폐렴", "폐결핵", "폐암", "폐기종", "천식",
        "호흡곤란", "호흡기", "흡인성 폐렴",
        "중증 급성 호흡기", "중동 호흡기", "코로나-19",
        "인플루엔자", "폐 혈전색전증"
    ]):
        dept = "호흡기내과"

    # -------------------------
    # 5) 소화기내과
    # -------------------------
    elif any(kw in lbl for kw in [
        "위염", "위장염", "장염", "대장", "소장", "간",
        "담낭", "담석", "담관", "췌장", "복부 통증",
        "복막염", "소화불량", "소화성궤양", "식도",
        "염증성 장질환", "장티푸스", "식중독",
        "덤핑 증후군", "대사 증후군", "지방간",
        "간경화", "간암", "간세포성 암종", "급성 바이러스성 간염",
        "노로바이러스 장염", "바이러스성 장염"
    ]):
        dept = "소화기내과"

    # -------------------------
    # 6) 내분비/당뇨/갑상선
    # -------------------------
    elif any(kw in lbl for kw in [
        "당뇨병", "제1형 당뇨병", "당뇨병성", "비만",
        "갑상선", "쿠싱", "말단비대증", "갈색세포종",
        "지질혈증", "대사성", "저혈당증", "고혈당",
    ]):
        dept = "내분비내과"

    # -------------------------
    # 7) 신장내과 / 비뇨기과
    # -------------------------
    elif any(kw in lbl for kw in [
        "신부전", "신부전증", "신장 결석", "신증후군",
        "요로결석", "요실금", "배뇨장애", "방광염", "방광암",
        "신우신염", "요관", "요도하열", "요독증", "신질환",
    ]):
        dept = "신장내과/비뇨기과"

    # -------------------------
    # 8) 혈액/종양내과
    # -------------------------
    elif any(kw in lbl for kw in [
        "백혈병", "림프종", "골수종", "골수증식종양",
        "골수형성이상증후군", "혈소판감소증", "범혈구감소증",
        "종양", "암", "육종", "전이 암",
    ]):
        dept = "혈액종양내과"

    # -------------------------
    # 9) 류마티스/면역
    # -------------------------
    elif any(kw in lbl for kw in [
        "류마티스 관절염", "전신 홍반성 낭창", "자가면역",
        "베체트", "혈관염", "강직성 척추염",
    ]):
        dept = "류마티스내과"

    # -------------------------
    # 10) 정형외과 / 재활의학과
    # -------------------------
    elif any(kw in lbl for kw in [
        "골절", "골다공증", "골감소증", "골관절염",
        "관절염", "척추측만증", "요추", "경추",
        "십자 인대 손상", "무릎", "어깨", "무지외반증",
        "거북목", "족저 근막염", "내반슬", "외반슬",
        "사각턱", "주걱턱", "턱관절", "연골무형성증",
    ]):
        dept = "정형외과"

    # -------------------------
    # 11) 산부인과
    # -------------------------
    elif any(kw in lbl for kw in [
        "임신", "산후", "산욕열", "조산", "난소",
        "자궁", "태반", "양수", "유산", "난소암",
        "다태아", "난산", "월경", "과다월경", "성조숙증",
        "자간전증", "자간증",
    ]):
        dept = "산부인과"

    # -------------------------
    # 12) 소아청소년과
    # -------------------------
    elif any(kw in lbl for kw in [
        "소아마비", "영아", "미숙아", "선천성", "묘성 증후군",
        "다운 증후군", "윌리암스 증후군", "프라더 윌리",
        "클라인펠터", "터너 증후군", "영아 돌연사",
        "주의력결핍 과잉행동장애", "성인 주의력 결핍 과잉행동장애",
    ]):
        dept = "소아청소년과"

    # -------------------------
    # 13) 안과
    # -------------------------
    elif any(kw in lbl for kw in [
        "각막염", "결막염", "녹내장", "망막", "백내장",
        "안구", "시신경염", "안검하수", "안면홍조"  # 일부는 피부과일 수도 있음
    ]):
        dept = "안과"

    # -------------------------
    # 14) 피부과
    # -------------------------
    elif any(kw in lbl for kw in [
        "피부", "습진", "무좀", "백선", "피부염",
        "여드름", "지루성 피부염", "두드러기", "티눈",
        "사마귀", "쥐젖", "켈로이드", "땀띠", "옻 중독",
        "옴", "모낭염", "모공성 각화증",
    ]):
        dept = "피부과"

    # -------------------------
    # 15) 이비인후과
    # -------------------------
    elif any(kw in lbl for kw in [
        "중이염", "외이도염", "비염", "부비동염",
        "편도선염", "후두암", "비출혈", "백일해",
        "인후두염", "상기도 감염",
    ]):
        dept = "이비인후과"

    # -------------------------
    # 16) 비뇨의학과/남성
    # -------------------------
    elif any(kw in lbl for kw in [
        "전립선", "고환", "포경", "잠복고환증",
        "남성 불임", "음낭수종",
    ]):
        dept = "비뇨의학과"

    # -------------------------
    # 17) 유방/외과
    # -------------------------
    elif any(kw in lbl for kw in [
        "유방암", "유방 종양", "유선염",
    ]):
        dept = "외과(유방외과)"

    # -------------------------
    # 18) 항문/대장항문외과
    # -------------------------
    elif any(kw in lbl for kw in [
        "항문암", "치질", "치루", "치열"  # 일부는 라벨에 없을 수도 있음
    ]):
        dept = "대장항문외과"

    # -------------------------
    # 19) 응급/외상 (외과계)
    # -------------------------
    elif any(kw in lbl for kw in [
        "외상", "열상", "압궤 손상", "화상",
    ]):
        dept = "외상외과/응급의학과"

    # -------------------------
    # 20) 기본값: 내과
    # -------------------------
    # dept는 이미 "내과"로 초기화 되어 있음

    # -------------------------
    # 중증도 기본은 "일반"에서 시작, 일부 라벨만 예외 처리
    # -------------------------

    # 응급
    if any(kw in lbl for kw in [
        "급성 심근경색증", "뇌출혈", "대동맥 박리",
        "패혈증", "아나필락시스", "호흡곤란증후군",
        "공기가슴증", "폐 혈전색전증",
        "발작", "영아 돌연사", "급성 호흡곤란",
        "파상풍", "독성 쇼크", "파종혈관내응고",
        "태반 조기 박리", "양수색전증"
    ]):
        severity = "응급"

    # 주의
    elif any(kw in lbl for kw in [
        "협심증", "심부전", "심근염", "복막염",
        "복강 내 출혈", "위장관출혈", "폐렴",
        "요로결석", "췌장염", "맹장염", "충수염",
        "신부전", "임신성 고혈압", "임신성 당뇨병",
        "전치 태반", "조기 양막 파열", "자간전증",
        "자궁 외 임신",
    ]):
        severity = "주의"

    LABEL2DEPT[lbl] = dept
    LABEL2SEVERITY[lbl] = severity

print("총 라벨 수:", len(all_labels))
print("진료과 종류:", set(LABEL2DEPT.values()))
print("중증도 분포:",
      {s: sum(1 for v in LABEL2SEVERITY.values() if v == s) for s in ["일반", "주의", "응급"]})

총 라벨 수: 583
진료과 종류: {'호흡기내과', '내분비내과', '산부인과', '피부과', '외과(유방외과)', '신경과', '신장내과/비뇨기과', '혈액종양내과', '소화기내과', '이비인후과', '류마티스내과', '내과', '비뇨의학과', '소아청소년과', '정신건강의학과', '외상외과/응급의학과', '정형외과', '안과', '순환기내과'}
중증도 분포: {'일반': 540, '주의': 27, '응급': 16}


In [68]:
# 자동 매핑 결과를 CSV로 내보내기
import pandas as pd

# LABEL2DEPT, LABEL2SEVERITY가 이미 만들어져 있다고 가정
rows = []
for lbl in all_labels:
    rows.append({
        "label": lbl,
        "department": LABEL2DEPT.get(lbl, "내과"),
        "severity": LABEL2SEVERITY.get(lbl, "일반"),
        "reviewed": 0,   # 나중에 검수했으면 1로 바꾸기
        "comment": "",   # 수정 이유나 메모용
    })

df_map = pd.DataFrame(rows)
df_map.to_csv("label_mapping_draft.csv", index=False, encoding="utf-8-sig")

df_map.head()

,label,department,severity,reviewed,comment
0,B형간염,소화기내과,일반,0,
1,C형간염,소화기내과,일반,0,
2,HIV 감염,내과,일반,0,
3,VDT 증후군,내과,일반,0,
4,가와사키병,내과,일반,0,


In [ ]:
# 사람이 수정한 CSV를 다시 읽어서 “최종 매핑” 만들기
# 엑셀에서 수정 후, 파일명을 예를 들어 label_mapping_final.csv 로 저장
import pandas as pd

df_final = pd.read_csv("mediq_labels_final.csv")

LABEL2DEPT = dict(zip(df_final["label"], df_final["department"]))
LABEL2SEVERITY = dict(zip(df_final["label"], df_final["severity"]))

# 검수 안 된게 남아있는지 체크 (reviewed == 0)
unreviewed = df_final[df_final.get("reviewed", 1) == 0]
print("검수 안 된 라벨 개수:", len(unreviewed))

In [15]:
# 학습 루프 세팅 (optimizer / scheduler)
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm
import os, time
import torch

# ----------------------------
# 설정
# ----------------------------
TOTAL_EPOCHS = 3          # 최종 목표(1epoch -> 3epoch까지)
START_EPOCH = 2           # 지금 1epoch 모델(kmbert_epoch1) 불러왔으니 2부터 시작
LR = 2e-5

# 중간 저장 폴더(재부팅/꺼짐 대비)
CKPT_DIR = "./kmbert_checkpoint_longrun"  # 기존에 있던 폴더 그대로 활용
os.makedirs(CKPT_DIR, exist_ok=True)

# ----------------------------
# optimizer / scheduler
# ----------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

num_training_steps = (TOTAL_EPOCHS - START_EPOCH + 1) * len(train_loader)
num_warmup_steps = int(0.1 * num_training_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

print("training steps:", num_training_steps, "warmup:", num_warmup_steps)

# ----------------------------
# 학습
# ----------------------------
global_step = 0
SAVE_EVERY_N_STEPS = 2000   # 1000~5000 사이로 취향껏. 자주 저장할수록 안전(느려짐 약간)

for epoch in range(START_EPOCH, TOTAL_EPOCHS + 1):
    model.train()
    total_loss = 0.0

    pbar = tqdm(train_loader, desc=f"epoch {epoch}", leave=True)

    for step, batch in enumerate(pbar, start=1):
        batch_gpu = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch_gpu)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()

        # (선택) 그래디언트 폭주 방지
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        global_step += 1

        # tqdm 표시
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        # ----------------------------
        # ✅ 중간 체크포인트 저장 (재부팅 대비)
        # ----------------------------
        if global_step % SAVE_EVERY_N_STEPS == 0:
            ckpt_path = os.path.join(CKPT_DIR, f"step_{global_step}_epoch_{epoch}")
            os.makedirs(ckpt_path, exist_ok=True)

            model.save_pretrained(ckpt_path)
            tokenizer.save_pretrained(ckpt_path)

            # optimizer/scheduler도 같이 저장하면 "완전 재개" 가능
            torch.save(
                {
                    "epoch": epoch,
                    "global_step": global_step,
                    "optimizer": optimizer.state_dict(),
                    "scheduler": scheduler.state_dict(),
                },
                os.path.join(ckpt_path, "trainer_state.pt")
            )

            print(f"\n[CKPT SAVED] {ckpt_path}")

    avg_loss = total_loss / len(train_loader)
    print(f"[epoch {epoch}] train loss = {avg_loss:.4f}")

    # (선택) 매 epoch 끝날 때도 저장(안전)
    epoch_path = os.path.join(CKPT_DIR, f"epoch_{epoch}_final")
    os.makedirs(epoch_path, exist_ok=True)
    model.save_pretrained(epoch_path)
    tokenizer.save_pretrained(epoch_path)
    torch.save(
        {
            "epoch": epoch,
            "global_step": global_step,
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
        },
        os.path.join(epoch_path, "trainer_state.pt")
    )
    print(f"[EPOCH CKPT SAVED] {epoch_path}")

training steps: 288996 warmup: 28899


epoch 2:   0%|          | 0/144498 [00:00<?, ?it/s]


[CKPT SAVED] ./kmbert_checkpoint_longrun\step_2000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_4000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_6000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_8000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_10000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_12000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_14000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_16000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_18000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_20000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_22000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_24000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_26000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_28000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_30000_epoch_2

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_32000_epoch_2

[CKPT SAVED] ./kmbert_check

epoch 3:   0%|          | 0/144498 [00:00<?, ?it/s]


[CKPT SAVED] ./kmbert_checkpoint_longrun\step_146000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_148000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_150000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_152000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_154000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_156000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_158000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_160000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_162000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_164000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_166000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_168000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_170000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_172000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_174000_epoch_3

[CKPT SAVED] ./kmbert_checkpoint_longrun\step_176000_epoch_3

[CKPT S

In [16]:
save_dir = "./kmbert_epoch3"   # 새 폴더 이름 (원하는 이름으로 바꿔도 됨)

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("저장 완료:", save_dir)

저장 완료: ./kmbert_epoch3


In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_dir = "./kmbert_epoch3"   # 방금 저장한 폴더
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)

id2label = model.config.id2label
label2id = model.config.label2id

print("num labels:", len(id2label))

num labels: 583


In [18]:
import torch.nn.functional as F

def predict_topk(text, k=5):
    model.eval()
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)[0]

    topk = torch.topk(probs, k)
    results = [
        (id2label[idx.item()], prob.item())
        for idx, prob in zip(topk.indices, topk.values)
    ]
    return results

In [19]:
tests = [
    "나 살이 잘렸어",
    "아랫배가 아파요",
    "임신 8주차인데 아랫배가 아파요",
    "며칠째 기침하고 숨쉬기 힘들어요",
    "눈이 너무 뻑뻑하고 따가워요",
    "신물이 올라오고 속이 쓰려요",
]

for t in tests:
    print("입력:", t)
    preds = predict_topk(t, k=5)
    for lbl, p in preds:
        print(f"  - {lbl}: {p:.3f}")
    print("-" * 50)

입력: 나 살이 잘렸어
  - 두드러기: 0.134
  - 당뇨병: 0.060
  - 죽상경화증: 0.058
  - 쿠싱 증후군: 0.050
  - 섭식 장애: 0.043
--------------------------------------------------
입력: 아랫배가 아파요
  - 난소의 양성 종양: 0.547
  - 골반 염증성 질환: 0.058
  - 월경전 증후군: 0.056
  - 난소난관염: 0.042
  - 난소, 난관 이상: 0.025
--------------------------------------------------
입력: 임신 8주차인데 아랫배가 아파요
  - 유산: 0.787
  - 조기 진통: 0.037
  - 습관성 유산: 0.032
  - 조산: 0.028
  - 자궁 내 성장 지연: 0.009
--------------------------------------------------
입력: 며칠째 기침하고 숨쉬기 힘들어요
  - 중증 급성 호흡기 증후군: 0.138
  - 심낭염: 0.132
  - 기도 폐쇄: 0.115
  - 지역사회성 폐렴: 0.047
  - 천식: 0.043
--------------------------------------------------
입력: 눈이 너무 뻑뻑하고 따가워요
  - 안구 건조증: 0.996
  - 결막염: 0.001
  - 피부염: 0.000
  - 각막염: 0.000
  - 복시: 0.000
--------------------------------------------------
입력: 신물이 올라오고 속이 쓰려요
  - 식도염: 0.544
  - 급성 신우신염: 0.183
  - 만성 위염: 0.125
  - 두통: 0.019
  - 만성 신우신염: 0.012
--------------------------------------------------
